# Multi-Agent P&C Claims Triage (LangChain + LangGraph)

This notebook is organized so that **each tool** and **each agent** is defined in its **own separate cell** for clarity and easy debugging.

Workflow:
**Intake → Coverage → Fraud → Estimator → Decision**

Outputs:
- Fraud confidence score (0–1)
- Decision: **approve / deny / refer_siu**
- Approved amount when applicable


## 1) Install dependencies



In [21]:
!pip -q uninstall -y langgraph-prebuilt langgraph langchain langchain-core langchain-openai langchain-community || true


!pip -q install -U \
  "pydantic==2.12.3" \
  "langgraph==1.0.8" \
  "langchain==1.2.9" \
  "langchain-openai==1.1.8" \
  "requests==2.32.4"


## 2) Set API key

In [22]:
import os
os.environ["OPENAI_API_KEY"] = "GET_UR_KEY"


## 3) Sanity checks

In [23]:
import importlib.metadata as md

def v(pkg: str):
    try:
        return md.version(pkg)
    except md.PackageNotFoundError:
        return "NOT INSTALLED"

print("langgraph:", v("langgraph"))
print("langgraph-prebuilt:", v("langgraph-prebuilt"))
print("langchain:", v("langchain"))
print("langchain-core:", v("langchain-core"))
print("langchain-openai:", v("langchain-openai"))
print("pydantic:", v("pydantic"))

from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent
from langgraph.graph import StateGraph, END

print("Imports OK")


langgraph: 1.0.8
langgraph-prebuilt: 1.0.7
langchain: 1.2.9
langchain-core: 1.2.11
langchain-openai: 1.1.8
pydantic: 2.12.3
Imports OK


## 4) Dummy data (policies, customers, vehicles, claims)

In [24]:
from typing import Dict, Any
from datetime import date

DUMMY_POLICIES: Dict[str, Dict[str, Any]] = {
    "POL1001": {
        "policy_id": "POL1001",
        "customer_id": "CUST01",
        "status": "ACTIVE",
        "effective_date": "2025-01-01",
        "expiry_date": "2026-01-01",
        "lines": {
            "AUTO": {
                "liability_limit": 100000,
                "collision_deductible": 500,
                "comprehensive_deductible": 250,
                "collision_limit": 40000,
                "comprehensive_limit": 20000,
                "excluded_drivers": [],
            },
            "PROPERTY": {
                "dwelling_limit": 300000,
                "contents_limit": 75000,
                "property_deductible": 1000,
                "exclusions": ["FLOOD"],
            },
        },
        "payment_history_ok": True,
        "prior_fraud_flag": False,
    },
    "POL2002": {
        "policy_id": "POL2002",
        "customer_id": "CUST02",
        "status": "ACTIVE",
        "effective_date": "2025-06-01",
        "expiry_date": "2026-06-01",
        "lines": {
            "AUTO": {
                "liability_limit": 50000,
                "collision_deductible": 1000,
                "comprehensive_deductible": 500,
                "collision_limit": 25000,
                "comprehensive_limit": 10000,
                "excluded_drivers": ["DRIVER_X"],
            }
        },
        "payment_history_ok": True,
        "prior_fraud_flag": True,
    },
}

DUMMY_CUSTOMERS: Dict[str, Dict[str, Any]] = {
    "CUST01": {
        "customer_id": "CUST01",
        "name": "Alex Morgan",
        "tenure_years": 6,
        "risk_tier": "LOW",
        "address_zip": "07030",
        "prior_claims_count": 1,
        "prior_claims": [
            {"claim_id": "CLM-OLD-01", "date": "2022-11-02", "type": "AUTO_COLLISION", "paid": 1800}
        ],
    },
    "CUST02": {
        "customer_id": "CUST02",
        "name": "Jordan Lee",
        "tenure_years": 1,
        "risk_tier": "HIGH",
        "address_zip": "11218",
        "prior_claims_count": 5,
        "prior_claims": [
            {"claim_id": "CLM-OLD-11", "date": "2024-02-14", "type": "AUTO_THEFT", "paid": 9000},
            {"claim_id": "CLM-OLD-12", "date": "2024-09-01", "type": "AUTO_GLASS", "paid": 450},
            {"claim_id": "CLM-OLD-13", "date": "2025-01-20", "type": "AUTO_COLLISION", "paid": 3200},
            {"claim_id": "CLM-OLD-14", "date": "2025-03-11", "type": "AUTO_COLLISION", "paid": 4100},
            {"claim_id": "CLM-OLD-15", "date": "2025-05-02", "type": "AUTO_FIRE", "paid": 12000},
        ],
    },
}

DUMMY_VEHICLES = {
    ("CUST01", "VIN001"): {"vin": "VIN001", "year": 2021, "make": "Toyota", "model": "Camry", "estimated_value": 22000},
    ("CUST02", "VIN777"): {"vin": "VIN777", "year": 2018, "make": "Honda", "model": "Civic", "estimated_value": 12000},
}

DUMMY_CLAIMS: Dict[str, Dict[str, Any]] = {
    "CLM1001": {
        "claim_id": "CLM1001",
        "policy_id": "POL1001",
        "loss_date": "2025-12-20",
        "reported_date": "2025-12-21",
        "lob": "AUTO",
        "claim_type": "AUTO_COLLISION",
        "location_zip": "07030",
        "driver_id": "DRIVER_A",
        "vehicle_vin": "VIN001",
        "description": "Rear-ended at a stop light. Moderate rear bumper damage and trunk misalignment.",
        "photos_provided": True,
        "police_report": True,
        "injury_reported": False,
        "estimated_repair_cost": 4200,
        "repair_shop": "Hudson Auto Body",
    },
    "CLM2002": {
        "claim_id": "CLM2002",
        "policy_id": "POL2002",
        "loss_date": "2025-06-03",
        "reported_date": "2025-06-20",
        "lob": "AUTO",
        "claim_type": "AUTO_THEFT",
        "location_zip": "11218",
        "driver_id": "DRIVER_X",
        "vehicle_vin": "VIN777",
        "description": "Vehicle stolen overnight. No security footage. Keys were 'misplaced'.",
        "photos_provided": False,
        "police_report": True,
        "injury_reported": False,
        "estimated_repair_cost": 0,
        "repair_shop": None,
    },
}


## 5) Tools

Each tool is defined in its own cell below.

### Tool: get_policy

In [48]:
from langchain_core.tools import tool
from typing import Dict, Any

@tool
def get_policy(policy_id: str) -> Dict[str, Any]:
    """Fetch a policy record by policy_id."""
    return DUMMY_POLICIES.get(policy_id, {"error": f"Policy {policy_id} not found"})


### Tool: get_customer

In [49]:
from langchain_core.tools import tool
from typing import Dict, Any

@tool
def get_customer(customer_id: str) -> Dict[str, Any]:
    """Fetch a customer record by customer_id."""
    return DUMMY_CUSTOMERS.get(customer_id, {"error": f"Customer {customer_id} not found"})


### Tool: get_claim

In [50]:
from langchain_core.tools import tool
from typing import Dict, Any

@tool
def get_claim(claim_id: str) -> Dict[str, Any]:
    """Fetch a claim record by claim_id."""
    return DUMMY_CLAIMS.get(claim_id, {"error": f"Claim {claim_id} not found"})


### Tool: get_vehicle

In [51]:
from langchain_core.tools import tool
from typing import Dict, Any

@tool
def get_vehicle(customer_id: str, vin: str) -> Dict[str, Any]:
    """Fetch a vehicle record (dummy) for valuation and context."""
    return DUMMY_VEHICLES.get((customer_id, vin), {"error": f"Vehicle {vin} for {customer_id} not found"})


### Tool: coverage_check

In [52]:
from langchain_core.tools import tool
from typing import Dict, Any, Optional

@tool
def coverage_check(policy_id: str, lob: str, claim_type: str, driver_id: Optional[str] = None) -> Dict[str, Any]:
    """Validate coverage eligibility: active policy, line exists, excluded driver, simple exclusions."""
    pol = DUMMY_POLICIES.get(policy_id)
    if not pol:
        return {"covered": False, "reason": "policy_not_found"}
    if pol.get("status") != "ACTIVE":
        return {"covered": False, "reason": "policy_not_active"}

    lines = pol.get("lines", {})
    if lob not in lines:
        return {"covered": False, "reason": f"lob_{lob}_not_on_policy"}

    if lob == "AUTO" and driver_id:
        excluded = set(lines["AUTO"].get("excluded_drivers", []))
        if driver_id in excluded:
            return {"covered": False, "reason": "excluded_driver"}

    if lob == "PROPERTY":
        exclusions = set(lines["PROPERTY"].get("exclusions", []))
        if "FLOOD" in exclusions and "FLOOD" in claim_type.upper():
            return {"covered": False, "reason": "excluded_flood"}

    return {"covered": True, "reason": "eligible"}


### Tool: estimate_payout

In [53]:
from langchain_core.tools import tool
from typing import Dict, Any

@tool
def estimate_payout(policy_id: str, claim_id: str) -> Dict[str, Any]:
    """Estimate payable amount using simple limits/deductible logic."""
    pol = DUMMY_POLICIES.get(policy_id)
    clm = DUMMY_CLAIMS.get(claim_id)
    if not pol or not clm:
        return {"error": "policy_or_claim_not_found"}

    lob = clm["lob"]
    if lob == "AUTO":
        auto = pol["lines"]["AUTO"]
        ctype = clm["claim_type"]
        if ctype == "AUTO_COLLISION":
            deductible = auto["collision_deductible"]
            limit = auto["collision_limit"]
            base = clm.get("estimated_repair_cost", 0)
            payable = max(0, min(limit, base) - deductible)
            return {"payable": payable, "deductible": deductible, "limit": limit, "basis": "repair_cost_minus_deductible"}

        if ctype == "AUTO_THEFT":
            deductible = auto["comprehensive_deductible"]
            limit = auto["comprehensive_limit"]
            customer_id = pol["customer_id"]
            vin = clm.get("vehicle_vin")
            vehicle = DUMMY_VEHICLES.get((customer_id, vin), {})
            acv = vehicle.get("estimated_value", 0)
            payable = max(0, min(limit, acv) - deductible)
            return {"payable": payable, "deductible": deductible, "limit": limit, "acv": acv, "basis": "acv_capped_minus_deductible"}

    return {"error": f"Unsupported LOB/type: {lob}/{clm.get('claim_type')}"}


### Tool: fraud_score

In [54]:
from langchain_core.tools import tool
from typing import Dict, Any
from datetime import date

@tool
def fraud_score(policy_id: str, claim_id: str) -> Dict[str, Any]:
    """Return a fraud confidence score (0-1) using simple heuristics + explainable signals."""
    pol = DUMMY_POLICIES.get(policy_id, {})
    clm = DUMMY_CLAIMS.get(claim_id, {})
    if "policy_id" not in pol or "claim_id" not in clm:
        return {"error": "policy_or_claim_not_found"}

    signals = []
    score = 0.05

    # Late reporting
    try:
        ld = date.fromisoformat(clm["loss_date"])
        rd = date.fromisoformat(clm["reported_date"])
        delay_days = (rd - ld).days
    except Exception:
        delay_days = 0
    if delay_days >= 10:
        score += 0.20
        signals.append({"signal": "late_reporting", "weight": 0.20, "detail": f"delay_days={delay_days}"})

    if not clm.get("photos_provided", False):
        score += 0.15
        signals.append({"signal": "no_photos", "weight": 0.15})

    if clm.get("police_report") is False:
        score += 0.15
        signals.append({"signal": "no_police_report", "weight": 0.15})

    if pol.get("prior_fraud_flag"):
        score += 0.20
        signals.append({"signal": "prior_fraud_flag_on_policy", "weight": 0.20})

    cust = DUMMY_CUSTOMERS.get(pol["customer_id"], {})
    if cust.get("prior_claims_count", 0) >= 4:
        score += 0.15
        signals.append({"signal": "high_prior_claims_count", "weight": 0.15, "detail": f"count={cust.get('prior_claims_count')}"})

    if clm.get("claim_type") == "AUTO_THEFT":
        desc = (clm.get("description") or "").lower()
        if "keys" in desc and "misplaced" in desc:
            score += 0.15
            signals.append({"signal": "keys_misplaced_theft_pattern", "weight": 0.15})

    auto = pol.get("lines", {}).get("AUTO", {})
    if clm.get("driver_id") in set(auto.get("excluded_drivers", [])):
        score += 0.30
        signals.append({"signal": "excluded_driver_in_claim", "weight": 0.30})

    score = max(0.0, min(1.0, score))
    band = "high" if score >= 0.70 else "medium" if score >= 0.40 else "low"
    return {"fraud_confidence": score, "fraud_band": band, "signals": signals}


## 6) Agents

Each agent is created in its own cell below

### Agent: Intake Agent

In [32]:
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent

MODEL = "gpt-4o-mini"
llm = ChatOpenAI(model=MODEL, temperature=0.2)

INTAKE_TOOLS = [get_claim, get_policy, get_customer, get_vehicle]

intake_agent = create_react_agent(
    llm,
    tools=INTAKE_TOOLS,
    prompt=(
        "You are the Intake Agent for a P&C claims system.\n"
        "Goal: summarize the claim, extract structured facts, and identify missing info.\n"
        "Use tools to fetch claim/policy/customer/vehicle data.\n"
        "Output JSON with keys: summary, facts, missing_info, next_questions."
    ),
)

print("Intake agent created")


Intake agent created


/tmp/ipython-input-3013830525.py:9: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  intake_agent = create_react_agent(


### Agent: Coverage Agent

In [33]:
from langgraph.prebuilt import create_react_agent

COVERAGE_TOOLS = [get_claim, get_policy, coverage_check]

coverage_agent = create_react_agent(
    llm,
    tools=COVERAGE_TOOLS,
    prompt=(
        "You are the Coverage Agent.\n"
        "Goal: determine whether the claim is eligible for coverage under the policy.\n"
        "Use coverage_check and policy/claim tools.\n"
        "Output JSON with keys: covered (true/false), reason, deductible, limits, notes."
    ),
)

print("Coverage agent created")


Coverage agent created


/tmp/ipython-input-3500581910.py:5: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  coverage_agent = create_react_agent(


### Agent: Fraud Agent

In [34]:
from langgraph.prebuilt import create_react_agent

FRAUD_TOOLS = [get_claim, get_policy, get_customer, fraud_score]

fraud_agent = create_react_agent(
    llm,
    tools=FRAUD_TOOLS,
    prompt=(
        "You are the Fraud Analyst Agent.\n"
        "Goal: produce a fraud confidence score (0-1) and rationale.\n"
        "Always call the fraud_score tool.\n"
        "Output JSON with keys: fraud_confidence, fraud_band, rationale, signals."
    ),
)

print("Fraud agent created")


Fraud agent created


/tmp/ipython-input-2054204198.py:5: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  fraud_agent = create_react_agent(


### Agent: Estimator Agent

In [35]:
from langgraph.prebuilt import create_react_agent

ESTIMATOR_TOOLS = [get_claim, get_policy, get_customer, get_vehicle, estimate_payout]

estimator_agent = create_react_agent(
    llm,
    tools=ESTIMATOR_TOOLS,
    prompt=(
        "You are the Damage Estimator Agent.\n"
        "Goal: estimate payable amount based on policy limits/deductible and claim details.\n"
        "Always call estimate_payout.\n"
        "Output JSON with keys: payable, deductible, limit, basis, notes."
    ),
)

print("Estimator agent created")


Estimator agent created


/tmp/ipython-input-3465099974.py:5: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  estimator_agent = create_react_agent(


### Agent: Decision Agent

In [36]:
from langgraph.prebuilt import create_react_agent

DECISION_TOOLS = [coverage_check, fraud_score, estimate_payout, get_claim, get_policy]

decision_agent = create_react_agent(
    llm,
    tools=DECISION_TOOLS,
    prompt=(
        "You are the Decision Agent (claims adjudication).\n"
        "Rules:\n"
        "1) If coverage is NOT eligible -> deny with reason.\n"
        "2) If fraud_confidence >= 0.70 -> refer to SIU (do NOT approve payment).\n"
        "3) Otherwise approve and return payable amount.\n"
        "Output JSON with keys: decision (approve/deny/refer_siu), reason, approved_amount, fraud_confidence."
    ),
)

print("Decision agent created")


Decision agent created


/tmp/ipython-input-1221645638.py:5: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  decision_agent = create_react_agent(


## 7) Orchestrate the workflow with LangGraph

Note: node names must not collide with state keys.

In [37]:
from typing import TypedDict, Any
from langchain_core.messages import HumanMessage
from langgraph.graph import StateGraph, END

class ClaimsState(TypedDict, total=False):
    claim_id: str
    policy_id: str
    intake: Any
    coverage: Any
    fraud: Any
    estimate: Any
    decision: Any

def _invoke_agent(agent, user_text: str):
    return agent.invoke({"messages": [HumanMessage(content=user_text)]})

def intake_node(state: ClaimsState) -> ClaimsState:
    claim_id = state["claim_id"]
    out = _invoke_agent(intake_agent, f"Process claim intake for claim_id={claim_id}. Use tools and return JSON.")
    return {"intake": out}

def coverage_node(state: ClaimsState) -> ClaimsState:
    claim_id = state["claim_id"]
    out = _invoke_agent(coverage_agent, f"Validate coverage for claim_id={claim_id}. Use tools; return JSON.")
    return {"coverage": out}

def fraud_node(state: ClaimsState) -> ClaimsState:
    claim_id = state["claim_id"]
    out = _invoke_agent(fraud_agent, f"Assess fraud for claim_id={claim_id}. You MUST call fraud_score. Return JSON.")
    return {"fraud": out}

def estimator_node(state: ClaimsState) -> ClaimsState:
    claim_id = state["claim_id"]
    out = _invoke_agent(estimator_agent, f"Estimate payout for claim_id={claim_id}. You MUST call estimate_payout. Return JSON.")
    return {"estimate": out}

def decision_node(state: ClaimsState) -> ClaimsState:
    claim_id = state["claim_id"]
    out = _invoke_agent(
        decision_agent,
        "Make final decision for claim_id={}. Use tools if needed. Consider coverage + fraud + estimate. Return JSON.".format(claim_id),
    )
    return {"decision": out}

def build_claims_graph():
    g = StateGraph(ClaimsState)
    g.add_node("intake_step", intake_node)
    g.add_node("coverage_step", coverage_node)
    g.add_node("fraud_step", fraud_node)
    g.add_node("estimate_step", estimator_node)
    g.add_node("decision_step", decision_node)

    g.set_entry_point("intake_step")
    g.add_edge("intake_step", "coverage_step")
    g.add_edge("coverage_step", "fraud_step")
    g.add_edge("fraud_step", "estimate_step")
    g.add_edge("estimate_step", "decision_step")
    g.add_edge("decision_step", END)
    return g.compile()

claims_app = build_claims_graph()
print("Claims graph compiled")


Claims graph compiled


## 8) Visualize the workflow

In [38]:
from IPython.display import HTML, display

mermaid = """
flowchart LR
  intake_step[Intake Agent]
  coverage_step[Coverage Agent]
  fraud_step[Fraud Agent]
  estimate_step[Estimator Agent]
  decision_step[Decision Agent]
  end((END))

  intake_step --> coverage_step --> fraud_step --> estimate_step --> decision_step --> end
"""

display(HTML(f"""
<script src="https://cdn.jsdelivr.net/npm/mermaid@10/dist/mermaid.min.js"></script>
<div class="mermaid">
{mermaid}
</div>
<script>
  mermaid.initialize({{ startOnLoad: true }});
</script>
"""))


## 9) Run the simulation

In [39]:
def run_claim(claim_id: str):
    claim = DUMMY_CLAIMS.get(claim_id)
    if not claim:
        raise ValueError(f"Unknown claim_id: {claim_id}")
    state: ClaimsState = {"claim_id": claim_id, "policy_id": claim["policy_id"]}
    return claims_app.invoke(state)

out1 = run_claim("CLM1001")
out2 = run_claim("CLM2002")

list(out1.keys()), list(out2.keys())


(['claim_id',
  'policy_id',
  'intake',
  'coverage',
  'fraud',
  'estimate',
  'decision'],
 ['claim_id',
  'policy_id',
  'intake',
  'coverage',
  'fraud',
  'estimate',
  'decision'])

### Print decisions

In [40]:
def last_message_text(agent_output):
    try:
        msgs = agent_output.get("messages", [])
        if msgs:
            return msgs[-1].content
    except Exception:
        pass
    return str(agent_output)

print("===== CLM1001 (expected: approve) =====")
print(last_message_text(out1["decision"]))
print("\n===== CLM2002 (expected: refer SIU / deny if excluded) =====")
print(last_message_text(out2["decision"]))


===== CLM1001 (expected: approve) =====
```json
{
  "decision": "approve",
  "reason": "Claim is eligible for coverage and fraud confidence is low.",
  "approved_amount": 3700,
  "fraud_confidence": 0.05
}
```

===== CLM2002 (expected: refer SIU / deny if excluded) =====
```json
{
  "decision": "deny",
  "reason": "excluded_driver",
  "approved_amount": 0,
  "fraud_confidence": 1.0
}
```
